In [1]:
import tensorflow as tf
from tensorflow import keras

# !pip install tensorflow-datasets
import tensorflow_datasets as tfds

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(f"Tensorflow version {tf.__version__}")

c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tensorflow version 2.13.0


In [2]:
# Download project data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv -nv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv -nv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

2024-01-07 16:38:02 URL:https://cdn.freecodecamp.org/project-data/sms/train-data.tsv [358233/358233] -> "train-data.tsv" [1]
2024-01-07 16:38:03 URL:https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv [118774/118774] -> "valid-data.tsv" [1]


In [3]:
train_data = pd.read_csv(train_file_path, sep='\t', names=['y', 'x'])
test_data = pd.read_csv(test_file_path, sep='\t', names=['y', 'x'])

train_data.y.replace(['ham', 'spam'], ['0', '1'], inplace=True)
test_data.y.replace(['ham', 'spam'], ['0', '1'], inplace=True)

os.remove(train_file_path)
os.remove(test_file_path)

In [4]:
train_data.head(8)

,y,x
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...
5,0,in xam hall boy asked girl tell me the startin...
6,0,genius what's up. how your brother. pls send h...
7,0,they finally came to fix the ceiling.


In [5]:
train_data = train_data.astype({'y': 'int32', 'x': 'string'})
test_data = test_data.astype({'y': 'int32', 'x': 'string'})

print(f'Train dtypes:\n{train_data.dtypes}\n')
print(f'Test dtypes:\n{test_data.dtypes}')

Train dtypes:
y             int32
x    string[python]
dtype: object

Test dtypes:
y             int32
x    string[python]
dtype: object


In [6]:
max_vocab = 6000
text_length = 80

vectorization = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab,
    standardize='lower_and_strip_punctuation',
    output_mode="int",
    output_sequence_length=text_length,
)

vectorization.adapt(pd.concat([train_data.x, test_data.x]))

In [7]:
vectorization("hi how are you brother")

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([100,  54,  21,   4, 618,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=int64)>

In [8]:
model = tf.keras.models.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorization)
 # Input dim is +1 because it's the vocabulary + the 0 token for non-vocabulary words
model.add(tf.keras.layers.Embedding(input_dim=max_vocab + 1, output_dim=32))
model.add(tf.keras.layers.LSTM(15, dropout=0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 80)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 80, 32)            192032    
                                                                 
 lstm (LSTM)                 (None, 15)                2880      
                                                                 
 dense (Dense)               (None, 1)                 16        
                                                                 
Total params: 194928 (761.44 KB)
Trainable params: 194928 (761.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.fit(x=train_data.x, y=train_data.y, epochs=20, validation_data=(test_data.x, test_data.y))

Epoch 1/20
131/131 [==============================] - 4s 16ms/step - loss: 0.7020 - accuracy: 0.1744 - val_loss: 0.6983 - val_accuracy: 0.1358
Epoch 2/20
131/131 [==============================] - 2s 13ms/step - loss: 0.6947 - accuracy: 0.4171 - val_loss: 0.6909 - val_accuracy: 0.8657
Epoch 3/20
131/131 [==============================] - 2s 13ms/step - loss: 0.6873 - accuracy: 0.7633 - val_loss: 0.6837 - val_accuracy: 0.8657
Epoch 4/20
131/131 [==============================] - 2s 13ms/step - loss: 0.6799 - accuracy: 0.8615 - val_loss: 0.6764 - val_accuracy: 0.8657
Epoch 5/20
131/131 [==============================] - 2s 13ms/step - loss: 0.6727 - accuracy: 0.8660 - val_loss: 0.6689 - val_accuracy: 0.8657
Epoch 6/20
131/131 [==============================] - 2s 13ms/step - loss: 0.6650 - accuracy: 0.8660 - val_loss: 0.6609 - val_accuracy: 0.8657
Epoch 7/20
131/131 [==============================] - 2s 13ms/step - loss: 0.6568 - accuracy: 0.8660 - val_loss: 0.6523 - val_accuracy: 0.8657

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
